Using KNN SVM Gaussian Naïve Bayes and fusing their output and feeding into a random forest model which predicts the outcome

In [4]:
# Import dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import arff as arf

In [5]:
file = open("Dataset/final-dataset.arff")
decoder = arf.ArffDecoder()
data=decoder.decode(file,encode_nominal=True)

Break the data into main data and labels

main data - logs which we will input to our model

In [6]:
vals = [val[0: -1] for val in data['data']]
labels = [lab[-1] for lab in data['data']]

In [8]:
da =  set(labels)
# getting 600 for each category
brac = 600
templ = []
tempd = []
for i in da:
    coun = 0
    while coun <brac:
        # filtering the data or preprocessing the data using labels
        for j in range(len(labels)):
            if labels[j]:
                templ.append(labels[j])
                tempd.append(vals[j])
                coun = coun + 1
            if coun == brac:
                break
vals = tempd
labels = templ


In [9]:
l = len(vals)
print(l)

3000


In [10]:
# split the data into training and test set
X_train, X_test, Y_train, Y_test = train_test_split(vals, labels, stratify=labels, test_size=0.2, random_state=0)

In [11]:
# Scaling
scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)
y_train = np.array(Y_train)
y_test = np.array(Y_test)


Training our first Model 
1. SVM

In [12]:
model = SVC(kernel='sigmoid', gamma='auto')
model.fit(x_train, y_train)

SVC(gamma='auto', kernel='sigmoid')

In [13]:
# get predictions from it
y_pred = model.predict(x_test)

In [14]:
# print accuracy
print((accuracy_score(y_pred, y_test))*100,"%")

88.66666666666667 %


2. KNN

In [15]:
model1 = KNeighborsClassifier(n_neighbors=5)
model1.fit(x_train, y_train)

KNeighborsClassifier()

In [16]:
y_pred1 = model1.predict(x_test)

In [17]:
print(accuracy_score(y_pred1, y_test)*100, "%")

98.16666666666667 %


3. Gaussian Naive bayes model

In [19]:
model2 = GaussianNB()
model2.fit(x_train, y_train)

GaussianNB()

In [20]:
y_pred2 = model2.predict(x_test)

In [21]:
print((accuracy_score(y_pred2, y_test))*100, "%")

90.16666666666666 %


In [22]:
# Split the training data into further parts so that we can fuse and feed whole data into Random forest classifer
train_x, val_x, train_y, val_y = train_test_split(x_train, y_train, stratify=y_train, test_size=0.2, random_state=0)

In [23]:
print(x_train.shape, x_test.shape)

(2400, 27) (600, 27)


# Fusing all three models

In [30]:
# creating coumn variable which will hold the name of columns in our dataset
columns = ['SRC_ADD', 'DES_ADD', 'PKT_ID', 'FROM_NODE', 'TO_NODE', 'PKT_TYPE', 'PKT_SIZE', 'FLAGS', 'FID', 'SEQ_NUMBER', 'NUMBEROFPKT',
            'NUMBEROFBYTE', 'NODENAMEFROM', 'NODENAMETO', 'PKT_IN', 'PKT_OUT', 'PKT_R', 'PKTDELAYNODE', 'PKT_RATE', 'BYTE_RATE',
                'PKTAVGSIZE', 'UTILIZATION', 'PKT_DELAY', 'PKTSENDTIME', 'PKTRESEVEDTIME', 'FIRSTPKTSENT', 'LASTPKTRESEVED']

In [24]:
model1 = SVC(kernel='sigmoid', gamma='auto')
model1.fit(train_x, train_y)
y_val_pred1 = model1.predict(val_x)
y_val_pred1 = pd.DataFrame(y_val_pred1)
y_test_pred1 = model1.predict(x_test)
y_test_pred1 = pd.DataFrame(y_test_pred1)

In [25]:
model2 = KNeighborsClassifier(n_neighbors=5)
model2.fit(train_x, train_y)
y_val_pred2 = model1.predict(val_x)
y_val_pred2 = pd.DataFrame(y_val_pred2)
y_test_pred2 = model1.predict(x_test)
y_test_pred2 = pd.DataFrame(y_test_pred2)

In [26]:
model3 = GaussianNB()
model3.fit(train_x, train_y)
y_val_pred3 = model1.predict(val_x)
y_val_pred3 = pd.DataFrame(y_val_pred3)
y_test_pred3 = model1.predict(x_test)
y_test_pred3 = pd.DataFrame(y_test_pred3)

In [31]:
val_input = pd.concat([pd.DataFrame(val_x, columns = columns), y_val_pred1, y_val_pred2, y_val_pred3], axis=1)
test_input = pd.concat([pd.DataFrame(x_test, columns = columns), y_test_pred1, y_test_pred2, y_test_pred3], axis=1)

In [32]:
model = RandomForestClassifier(n_estimators=200)
model.fit(val_input, val_y)

c:\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier(n_estimators=200)

In [33]:
print((model.score(test_input, y_test))*100, "%")

98.0 %


c:\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
